<a href="https://colab.research.google.com/github/shraddha-18/Astropy-tutorials/blob/main/astropy_day_1_Coordinates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install astropy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install astroquery

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import astropy

In [4]:
from astropy import units as u
from astropy.coordinates import SkyCoord, Distance
from astropy.io import fits
from astropy.table import QTable
from astropy.utils.data import download_file

# from astroquery.gaia import Gaia
from astroquery.gaia import Gaia

Gaia.ROW_LIMIT = 10000  #set the row limit for returned data

n Astropy, the most common way of representing and working with sky coordinates is to use the SkyCoord object (docs). A SkyCoord can be created directly from angles or arrays of angles with associated units, as demonstrated below.

To get started, let's assume that we want to create a SkyCoord object for the center of the open star cluster NGC 188 so that later we can query and retrieve stars that might be members of the cluster. Let's also assume, for now, that we already know the sky coordinates of the cluster to be (12.11, 85.26) degrees in the ICRS coordinate frame. The ICRS — sometimes referred to as "equatorial" or "J2000" coordinates (more info about the ICRS) — is currently the most common astronomical coordinate frame for stellar or extragalactic astronomy, and is the default coordinate frame for SkyCoord. Since we already know the ICRS position of NGC 188 (see above), we can create a SkyCoord object for the cluster by passing the data in to the SkyCoord initializer:

In [5]:
ngc188_center = SkyCoord(12.11*u.deg, 85.26*u.deg, frame = 'icrs')
ngc188_center

<SkyCoord (ICRS): (ra, dec) in deg
    (12.11, 85.26)>

In the above initializations, we passed in astropy.units.Quantity objects with angular units to specify the angular components of our sky coordinates. The SkyCoord initializer will also accept string-formatted coordinates either as separate strings for Right Ascension (RA) and Declination (Dec) or a single string. For example, if we have sexagesimal sky coordinate data: In this case, the representation of the data includes specifications of the units (the "hms" for "hour minute second", and the "dms" for "degrees minute second"

In [6]:
ngc188_center.ra


<Longitude 12.11 deg>

In [7]:
ngc188_center.dec

<Latitude 85.26 deg>

In [8]:
SkyCoord('00h48m26.4s', '85d15m36s', frame = 'icrs')

<SkyCoord (ICRS): (ra, dec) in deg
    (12.11, 85.26)>

Some string representations do not explicitly define units, so it is sometimes necessary to specify the units of the string coordinate data explicitly if they are not implicitly included:

In [9]:
SkyCoord('00:48:26.4 85:15:36', unit = (u.hour, u.deg), frame = 'icrs')

<SkyCoord (ICRS): (ra, dec) in deg
    (12.11, 85.26)>

For the SkyCoord initializations demonstrated above, we assumed that we already had the coordinate component values ready. If you do not know the coordinate values and the object you are interested in is in SESAME, you can also automatically look up and load coordinate values from the name of the object using the SkyCoord.from_name() class method1 (docs). Note, however, that this requires an internet connection. It is safe to skip this cell if you are not connected to the internet because we already defined the object ngc188_center in the cells above.

In [10]:
ngc188_center = SkyCoord.from_name(' NGC 188')

In [11]:
ngc188_center

<SkyCoord (ICRS): (ra, dec) in deg
    (11.798, 85.244)>

The SkyCoord object that we defined now has various ways of accessing the coordinate information contained within it. All SkyCoord objects have attributes that allow you to retrieve the coordinate component data, but the component names will change depending on the coordinate frame of the SkyCoord you have. In our examples we have created a SkyCoord in the ICRS frame, so the component names are lower-case abbreviations of Right Ascension, .ra, and Declination, .dec:

In [12]:
ngc188_center.ra, ngc188_center.dec

(<Longitude 11.798 deg>, <Latitude 85.244 deg>)

The SkyCoord component attributes (here ra and dec) return specialized Quantity-like objects that make working with angular data easier. While Quantity (docs) is a general class that represents numerical values and physical units of any kind, astropy.coordinates defines subclasses of Quantity that are specifically designed for working with angles, such as the Angle (docs) class. The Angle class then has additional, more specialized subclasses Latitude (docs) and Longitude (docs). These objects store angles, provide useful attributes to quickly convert to common angular units, and enable formatting the numerical values in various formats. For example, in a Jupyter notebook, these objects know how to represent themselves using LaTeX:

In [13]:
ngc188_center.ra

<Longitude 11.798 deg>

In [14]:
ngc188_center.dec

<Latitude 85.244 deg>

In [15]:
type(ngc188_center.ra), type(ngc188_center.dec)

(astropy.coordinates.angles.Longitude, astropy.coordinates.angles.Latitude)

With these objects, we can retrieve the coordinate components in different units using the Quantity.to() method:

In [16]:
ngc188_center.ra.to(u.hourangle)

<Longitude 0.78653333 hourangle>

In [17]:
ngc188_center.ra.to(u.hourangle), #see the difference due to additional comma at last

(<Longitude 0.78653333 hourangle>,)

In [18]:
ngc188_center.ra.to(u.radian)


<Longitude 0.20591395 rad>

In [19]:
ngc188_center.ra.to(u.radian), # see the difference due to additional comma at last

(<Longitude 0.20591395 rad>,)

In [20]:
ngc188_center.ra.to(u.degree)

<Longitude 11.798 deg>

In [21]:
ngc188_center.ra.to(u.degree), # see the difference due to additional comma at last

(<Longitude 11.798 deg>,)

In [22]:
ngc188_center.ra.hour, ngc188_center.ra.radian, ngc188_center.ra.degree

(0.7865333333333335, 0.205913945150291, 11.798)

We can also format the values into strings with specified units (docs), for example:

In [23]:
ngc188_center.ra.to_string( unit = u.hourangle, sep = ':', pad = True)

'00:47:11.52'


## Querying the Gaia Archive to Retrieve Coordinates of Stars in NGC 188




Now that we have a SkyCoord object for the center of NGC 188, we can use this object with the astroquery package to query many different astronomical databases (see a full list of available services in the astroquery documentation). Here, we will use the SkyCoord object ngc188_center to select sources from the Gaia Data Release 2 catalog around the position of the center of NGC 188 to look for stars that might be members of the star cluster. To do this, we will use the astroquery.gaia subpackage (docs).

This requires an internet connection, but if it fails, the catalog file is included in the repository so you can load it locally (skip the next cell if you do not have an internet connection):

In [24]:
job = Gaia.cone_search_async( ngc188_center, radius = 0.5*u.deg)
ngc188_table = job.get_results()

INFO: Query finished. [astroquery.utils.tap.core]


In [25]:
ngc188_table

solution_id,DESIGNATION,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,astrometric_weight_al,astrometric_pseudo_colour,astrometric_pseudo_colour_error,mean_varpi_factor_al,astrometric_matched_observations,visibility_periods_used,astrometric_sigma5d_max,frame_rotator_object_type,matched_observations,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_nb_transits,rv_template_teff,rv_template_logg,rv_template_fe_h,phot_variable_flag,l,b,ecl_lon,ecl_lat,priam_flags,teff_val,teff_percentile_lower,teff_percentile_upper,a_g_val,a_g_percentile_lower,a_g_percentile_upper,e_bp_min_rp_val,e_bp_min_rp_percentile_lower,e_bp_min_rp_percentile_upper,flame_flags,radius_val,radius_percentile_lower,radius_percentile_upper,lum_val,lum_percentile_lower,lum_percentile_upper,datalink_url,dist
,,,,yr,deg,mas,deg,mas,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,mas,,,,1 / mas2,1 / um,1 / um,,,,mas,,,,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,,mag,mag,mag,km / s,km / s,,K,log(cm.s**-2),dex,,deg,deg,deg,deg,,K,K,K,mag,mag,mag,mag,mag,mag,,solRad,solRad,solRad,solLum,solLum,solLum,,
int64,object,int64,int64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,int32,float32,float32,float64,float64,int16,bool,float32,float64,float64,float32,int16,int16,float32,int32,int16,bool,int32,float64,float64,float32,float32,int32,float64,float64,float32,float32,int32,float64,float64,float32,float32,float32,int16,float32,float32,float32,float64,float64,int32,float32,float32,float32,object,float64,float64,float64,float64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,float32,float32,float32,float32,float32,float32,object,float64
1635721458409799680,Gaia DR2 573941431864200576,573941431864200576,83123185,2015.5,11.830261815824414,0.0542902667793964,85.24362670022514,0.042072395333520175,0.5157883765292425,0.05479372380616342,9.413275,-2.467045722266196,0.10926752585727999,-1.050994537936408,0.07743635218193472,-0.1009901,0.1395926,-0.3704649,0.08886854,0.23210102,0.02040666,0.04040181,0.004346489,0.4120766,-0.14762878,218,0,218,0,-0.35325864,205.14085,0.0,0.0,31,False,4.5962696,1.557349869368139,0.01295388655778083,0.023302104,25,17,0.10796351,0,28,False,242,4271.6310905308155,2.8860520946580523,1480.095,16.611881,24,2102.957721598712,7.879459656949609,266.8911,17.044312,24,3106.570736913345,8.954349540413443,346.93427,16.031218,1.2195642,0,1.013094,0.43243027,0.5806637,--,--,0,--,--,--,NOT_AVAILABLE,122.83963638206116,22.372614390347948,78.85637288274647,65.17038794976227,100001,5237.1167,5012.258,5333.6665,0.253,0.1349,0.4671,0.124,0.0622,0.2312,200111,0.99452305,0.9588433,1.0857568,0.6703374,0.5158293,0.82484543,https://gea.esac.esa.int/data-server/datalink/links?ID=Gaia+DR2+573941431864200576,0.002694749156617476
1635721458409799680,Gaia DR2 573941431864201600,573941431864201600,1665699094,2015.5,11.775835255508628,0.03255906133591321,85.24193176004697,0.027086561349367863,0.577254365272

In [26]:
#only keep stars brighter than G=19 magnitude

ngc188_table = ngc188_table[ngc188_table['phot_g_mean_mag'] < 19*u.mag]

In [27]:
cols = [
    'source_id',
    'ra',
    'dec',
    'parallax',
    'parallax_error',
    'pmra',
    'pmdec',
    'radial_velocity',
    'phot_g_mean_mag',
    'phot_bp_mean_mag',
    'phot_rp_mean_mag'
]

In [28]:
ngc188_table

solution_id,DESIGNATION,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,astrometric_weight_al,astrometric_pseudo_colour,astrometric_pseudo_colour_error,mean_varpi_factor_al,astrometric_matched_observations,visibility_periods_used,astrometric_sigma5d_max,frame_rotator_object_type,matched_observations,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_nb_transits,rv_template_teff,rv_template_logg,rv_template_fe_h,phot_variable_flag,l,b,ecl_lon,ecl_lat,priam_flags,teff_val,teff_percentile_lower,teff_percentile_upper,a_g_val,a_g_percentile_lower,a_g_percentile_upper,e_bp_min_rp_val,e_bp_min_rp_percentile_lower,e_bp_min_rp_percentile_upper,flame_flags,radius_val,radius_percentile_lower,radius_percentile_upper,lum_val,lum_percentile_lower,lum_percentile_upper,datalink_url,dist
,,,,yr,deg,mas,deg,mas,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,mas,,,,1 / mas2,1 / um,1 / um,,,,mas,,,,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,,mag,mag,mag,km / s,km / s,,K,log(cm.s**-2),dex,,deg,deg,deg,deg,,K,K,K,mag,mag,mag,mag,mag,mag,,solRad,solRad,solRad,solLum,solLum,solLum,,
int64,object,int64,int64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,int32,float32,float32,float64,float64,int16,bool,float32,float64,float64,float32,int16,int16,float32,int32,int16,bool,int32,float64,float64,float32,float32,int32,float64,float64,float32,float32,int32,float64,float64,float32,float32,float32,int16,float32,float32,float32,float64,float64,int32,float32,float32,float32,object,float64,float64,float64,float64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,float32,float32,float32,float32,float32,float32,object,float64
1635721458409799680,Gaia DR2 573941431864200576,573941431864200576,83123185,2015.5,11.830261815824414,0.0542902667793964,85.24362670022514,0.042072395333520175,0.5157883765292425,0.05479372380616342,9.413275,-2.467045722266196,0.10926752585727999,-1.050994537936408,0.07743635218193472,-0.1009901,0.1395926,-0.3704649,0.08886854,0.23210102,0.02040666,0.04040181,0.004346489,0.4120766,-0.14762878,218,0,218,0,-0.35325864,205.14085,0.0,0.0,31,False,4.5962696,1.557349869368139,0.01295388655778083,0.023302104,25,17,0.10796351,0,28,False,242,4271.6310905308155,2.8860520946580523,1480.095,16.611881,24,2102.957721598712,7.879459656949609,266.8911,17.044312,24,3106.570736913345,8.954349540413443,346.93427,16.031218,1.2195642,0,1.013094,0.43243027,0.5806637,--,--,0,--,--,--,NOT_AVAILABLE,122.83963638206116,22.372614390347948,78.85637288274647,65.17038794976227,100001,5237.1167,5012.258,5333.6665,0.253,0.1349,0.4671,0.124,0.0622,0.2312,200111,0.99452305,0.9588433,1.0857568,0.6703374,0.5158293,0.82484543,https://gea.esac.esa.int/data-server/datalink/links?ID=Gaia+DR2+573941431864200576,0.002694749156617476
1635721458409799680,Gaia DR2 573941431864201600,573941431864201600,1665699094,2015.5,11.775835255508628,0.03255906133591321,85.24193176004697,0.027086561349367863,0.577254365272

In [29]:
ngc188_table[cols].write('gaia_results.fits', overwrite = True)

In [30]:
ngc188_table

solution_id,DESIGNATION,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,astrometric_weight_al,astrometric_pseudo_colour,astrometric_pseudo_colour_error,mean_varpi_factor_al,astrometric_matched_observations,visibility_periods_used,astrometric_sigma5d_max,frame_rotator_object_type,matched_observations,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_nb_transits,rv_template_teff,rv_template_logg,rv_template_fe_h,phot_variable_flag,l,b,ecl_lon,ecl_lat,priam_flags,teff_val,teff_percentile_lower,teff_percentile_upper,a_g_val,a_g_percentile_lower,a_g_percentile_upper,e_bp_min_rp_val,e_bp_min_rp_percentile_lower,e_bp_min_rp_percentile_upper,flame_flags,radius_val,radius_percentile_lower,radius_percentile_upper,lum_val,lum_percentile_lower,lum_percentile_upper,datalink_url,dist
,,,,yr,deg,mas,deg,mas,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,mas,,,,1 / mas2,1 / um,1 / um,,,,mas,,,,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,,mag,mag,mag,km / s,km / s,,K,log(cm.s**-2),dex,,deg,deg,deg,deg,,K,K,K,mag,mag,mag,mag,mag,mag,,solRad,solRad,solRad,solLum,solLum,solLum,,
int64,object,int64,int64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,int32,float32,float32,float64,float64,int16,bool,float32,float64,float64,float32,int16,int16,float32,int32,int16,bool,int32,float64,float64,float32,float32,int32,float64,float64,float32,float32,int32,float64,float64,float32,float32,float32,int16,float32,float32,float32,float64,float64,int32,float32,float32,float32,object,float64,float64,float64,float64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,float32,float32,float32,float32,float32,float32,object,float64
1635721458409799680,Gaia DR2 573941431864200576,573941431864200576,83123185,2015.5,11.830261815824414,0.0542902667793964,85.24362670022514,0.042072395333520175,0.5157883765292425,0.05479372380616342,9.413275,-2.467045722266196,0.10926752585727999,-1.050994537936408,0.07743635218193472,-0.1009901,0.1395926,-0.3704649,0.08886854,0.23210102,0.02040666,0.04040181,0.004346489,0.4120766,-0.14762878,218,0,218,0,-0.35325864,205.14085,0.0,0.0,31,False,4.5962696,1.557349869368139,0.01295388655778083,0.023302104,25,17,0.10796351,0,28,False,242,4271.6310905308155,2.8860520946580523,1480.095,16.611881,24,2102.957721598712,7.879459656949609,266.8911,17.044312,24,3106.570736913345,8.954349540413443,346.93427,16.031218,1.2195642,0,1.013094,0.43243027,0.5806637,--,--,0,--,--,--,NOT_AVAILABLE,122.83963638206116,22.372614390347948,78.85637288274647,65.17038794976227,100001,5237.1167,5012.258,5333.6665,0.253,0.1349,0.4671,0.124,0.0622,0.2312,200111,0.99452305,0.9588433,1.0857568,0.6703374,0.5158293,0.82484543,https://gea.esac.esa.int/data-server/datalink/links?ID=Gaia+DR2+573941431864200576,0.002694749156617476
1635721458409799680,Gaia DR2 573941431864201600,573941431864201600,1665699094,2015.5,11.775835255508628,0.03255906133591321,85.24193176004697,0.027086561349367863,0.577254365272

In [31]:
ngc188_table = QTable.read('gaia_results.fits')

In [32]:
ngc188_table

source_id,ra,dec,parallax,parallax_error,pmra,pmdec,radial_velocity,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag
,deg,deg,mas,mas,mas / yr,mas / yr,km / s,mag,mag,mag
int64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32
573941431864200576,11.830261815824414,85.24362670022514,0.5157883765292425,0.05479372380616342,-2.467045722266196,-1.050994537936408,nan,16.611881256103516,17.0443115234375,16.031217575073242
573941431864201600,11.775835255508628,85.24193176004697,0.5772543652723914,0.03540945862424966,-2.516351668894645,-0.8355545501783187,nan,15.76044750213623,16.158985137939453,15.20403003692627
573941431864197120,11.781434275329108,85.24714789288664,0.5984132325327813,0.08208147042492284,-2.690351051420921,-0.7210521514030971,nan,17.208106994628906,17.746112823486328,16.51482391357422
573941431864196096,11.796092281131761,85.24947781526936,0.5531336087156113,0.030428982635073338,-2.398870491478188,-0.7953811054060391,nan,15.372490882873535,15.72161865234375,14.859831809997559
573941328784984320,11.724758702696125,85.24364992210644,0.5082635115306529,0.025382825094664167,-2.0711502731011304,-1.1803518806591535,nan,14.243685722351074,14.48997688293457,13.822223663330078
573941324487641600,11.719951501560104,85.24436556846314,0.49615782194329944,0.024641889550915272,-2.7290188233882215,-0.6363330445687266,nan,14.756906509399414,15.114192008972168,14.207357406616211
573941427566934400,11.767783483373126,85.25045643192475,0.5854568254354121,0.0675799543338964,-2.5044780776391873,-0.6386327518922068,nan,16.85712242126465,17.418291091918945,16.138044357299805
573941328784984832,11.714628651927297,85.24276162173139,0.7116524311248976,0.03750185270586405,-0.26566221404999757,4.024849662318981,nan,15.525108337402344,15.909825325012207,14.960417747497559


In [33]:
len(ngc188_table)

4954

The returned astropy.table Table object now contains about 5000 stars from Gaia DR2 around the coordinate position of the center of NGC 188. Let's now construct a SkyCoord object with the results table. In the Gaia data archive, the ICRS coordinates of a source are given as column names "ra" and "dec":

In [34]:
ngc188_table['ra']

<Quantity [11.83026182, 11.77583526, 11.78143428, ...,  9.00733176,
            6.28374403, 16.03700724] deg>

In [35]:
ngc188_table['dec']

<Quantity [85.2436267 , 85.24193176, 85.24714789, ..., 84.81444405,
           85.08002617, 84.91080112] deg>

In [36]:
ngc188_table['phot_g_mean_mag']

<Quantity [16.611881 , 15.7604475, 17.208107 , ..., 18.839067 , 13.899115 ,
           18.526287 ] mag>

Note that, because the Gaia archive provides data tables with associated units, and we read this table using the QTable object (docs), the above table columns are represented as Quantity objects with units of degrees. Note also that these columns contain many (>5000!) coordinate values. We can pass these directly in to SkyCoord to get a single SkyCoord object to represent all of these coordinates:

In [37]:
ngc188_gaia_coords = SkyCoord(ngc188_table['ra'], ngc188_table['dec'])

In [38]:
ngc188_gaia_coords

<SkyCoord (ICRS): (ra, dec) in deg
    [(11.83026182, 85.2436267 ), (11.77583526, 85.24193176),
     (11.78143428, 85.24714789), ..., ( 9.00733176, 84.81444405),
     ( 6.28374403, 85.08002617), (16.03700724, 84.91080112)]>

In [39]:
ngc188_gaia_coords.ra.to(u.radian),

(<Longitude [0.20647702, 0.2055271 , 0.20562482, ..., 0.1572076 ,
             0.10967202, 0.27989858] rad>,)